In [5]:
import simpy
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ETAPA 1: EXTRAÇÃO DE DADOS

def get_process_times_from_csv(arquivo):
    """Lê um arquivo CSV Yamazumi, aplica um fator de calibração e extrai os tempos."""
    def converter_tempo_para_segundos(tempo_str):
        if pd.isna(tempo_str) or not isinstance(tempo_str, str): return 0
        parts = str(tempo_str).split(':')
        try:
            if len(parts) == 3: h, m, s = map(int, parts); return h * 3600 + m * 60 + s
            if len(parts) == 2: m, s = map(int, parts); return m * 60 + s
        except (ValueError, TypeError): return 0
        return 0

    try:
        if not os.path.exists(arquivo): return None
        df = pd.read_csv(arquivo, header=5, sep=';', encoding='latin1', on_bad_lines='skip')
        class_col = next((col for col in df.columns if 'Classifica' in col), None)
        if not class_col: return None
        total_row = df[df[class_col] == 'Total'].copy()
        if total_row.empty: return None

        start_col = df.columns.get_loc(class_col) + 1
        end_col = len(df.columns)
        try: end_col = df.columns.get_loc('Coluna1')
        except KeyError: pass
            
        colunas_postos = df.columns[start_col:end_col]
        tempos_totais = total_row[colunas_postos].dropna(axis=1, how='all')
        tempos_formatados = tempos_totais.melt(var_name='Posto', value_name='Tempo_str')
        
        # Aplica o fator de calibração aqui
        tempos_formatados['Tempo_segundos'] = tempos_formatados['Tempo_str'].apply(converter_tempo_para_segundos)
        
        return pd.Series(tempos_formatados.Tempo_segundos.values, index=tempos_formatados.Posto).to_dict()

    except Exception: return None

# --- PARÂMETROS GLOBAIS ---
# FATOR_DE_CALIBRACAO = 0.15 
NUM_OPERADORES_PERNA1 = 14  
NUM_OPERADORES_PERNA2 = 11  
HORAS_SIMULACAO = 8
META_PRODUCAO = 76
HORAS_META = 7.7 
TAKT_TIME_SEGUNDOS = 5.5 * 60



MODELOS_CSV ={
    "Accelo": {"baumuster": ["C951102", "C951104", "C951111"], "perna": 1, "tempos": "Yamazumi - Accelo.csv"},
    "Atego": {"baumuster":["C951500", "C951501", "C951511", "C951514", "C951530", "C951544"],"perna": 1, "tempos":"Yamazumi - Atego.csv"},
    "Atego (ATP)":{"baumuster": ["C968403", "C968114"], "perna": 1, "tempos":"Yamazumi - ATP.csv"}, 
    "Actros":{"baumuster": ["C963400", "C963403", "C963411", "C963414", "C963424", "C963425"], "perna": 2, "tempos":"Yamazumi - Actros.csv"},
    "Arocs": {"baumuster":["C964016", "C964216", "C964231", "C964416"], "perna": 2, "tempos":"Yamazumi - Arocs.csv"},
    "Axor (ATP +)":{"baumuster": ["C968150", "C968450", "C968453", "C968461", "C968475"], "perna":2, "tempos": "Yamazumi - Actros.csv"}
}

print("--- Carregando e Calibrando Dados de Processo ---")
#PROCESSOS_FABRICA = {modelo: get_process_times_from_csv(arquivo) for modelo, arquivo in MODELOS_CSV[modelo]["tempos"]}
PROCESSOS_FABRICA = {
    modelo: get_process_times_from_csv(dados["tempos"])
    for modelo, dados in MODELOS_CSV.items()
}
PROCESSOS_FABRICA = {k: v for k, v in PROCESSOS_FABRICA.items() if v}
print(PROCESSOS_FABRICA)
print(f"Dados para {len(PROCESSOS_FABRICA.keys())} modelos carregados com sucesso.")

print(PROCESSOS_FABRICA)


--- Carregando e Calibrando Dados de Processo ---
{'Accelo': {'Arrefec.': 316, 'Diesel': 253, 'Reaperto': 235, 'Estepe': 290, 'Pneu LD': 250, 'Pneu LE': 250, 'Aperto LD': 304, 'Aperto LE': 304, 'Grade': 270, 'MecÃ¢nica 1': 194, 'MecÃ¢nica 2': 194, 'ElÃ©trica 1 ': 357, 'ElÃ©trica 2': 357, 'Controle ': 265, 'Motorista': 266, 'Quis': 270}, 'Atego': {'Arrefec.': 314, 'Diesel': 314, 'Reaperto': 294, 'Estepe': 267, 'Pneu LD': 308, 'Pneu LE': 308, 'Aperto LD': 315, 'Aperto LE': 315, 'Grade': 234, 'MecÃ¢nica 1': 194, 'MecÃ¢nica 2': 194, 'ElÃ©trica 1 ': 360, 'ElÃ©trica 2': 360, 'Controle ': 275, 'Motorista': 266, 'Quis': 270}, 'Atego (ATP)': {'Arrefec.': 330, 'Diesel': 326, 'Reaperto': 557, '5Âª Roda': 724, 'Estepe': 319, 'Pneu LD': 256, 'Pneu LE': 256, 'Aperto LD': 323, 'Aperto LE': 323, 'Grade': 230, 'MecÃ¢nica 1': 194, 'MecÃ¢nica 2': 194, 'ElÃ©trica 1 ': 360, 'ElÃ©trica 2': 360, 'Controle ': 275, 'Motorista': 266, 'Quis': 270}, 'Actros': {'Abastecimento': 555, 'Passa-disÃ§o': 752, 'Chineleir

## TESTE da MELINA

ETAPA 1 - OBTENDO INFORMAÇÕES YAMAZUMI

In [ ]:
import pandas as pd
import os
import re
from unidecode import unidecode


# OBTENDO TEMPOS POR ATIVIDADES DOS ARQUIVOS YAMAZUMI 
def get_process_times_from_csv(arquivo):
    """Lê um CSV Yamazumi, extrai os tempos por atividade e retorna um dicionário {atividade: tempo_em_segundos}."""
    
    def converter_tempo_para_segundos(tempo_str):
        if pd.isna(tempo_str) or not isinstance(tempo_str, str):
            return 0
        parts = str(tempo_str).split(':')
        try:
            if len(parts) == 3:
                h, m, s = map(int, parts)
                return h * 3600 + m * 60 + s
            elif len(parts) == 2:
                m, s = map(int, parts)
                return m * 60 + s
        except Exception:
            return 0
        return 0

    if not os.path.exists(arquivo):
        return {}

    df = pd.read_csv(arquivo, header=5, sep=';', encoding='latin1', on_bad_lines='skip')

    def normalizar_coluna(col):
        col = unidecode(col)
        col = col.upper()
        col = re.sub(r"[\"\'().\-\/]", "", col)
        col = re.sub(r"\s+", "_", col)
        return col.strip("_")

    df.columns = [normalizar_coluna(col) for col in df.columns]

    class_col = next((col for col in df.columns if 'CLASSIFICA' in col), None)
    if not class_col:
        return {}

    total_row = df[df[class_col] == 'Total'].copy()
    if total_row.empty:
        return {}

    start_col = df.columns.get_loc(class_col) + 1
    end_col = len(df.columns)
    try:
        end_col = df.columns.get_loc('COLUNA1')
    except KeyError:
        pass

    colunas_postos = df.columns[start_col:end_col]
    tempos_totais = total_row[colunas_postos].dropna(axis=1, how='all')
    tempos_formatados = tempos_totais.melt(var_name='Atividade', value_name='Tempo_str')
    tempos_formatados['Tempo_segundos'] = tempos_formatados['Tempo_str'].apply(converter_tempo_para_segundos)

    return pd.Series(tempos_formatados.Tempo_segundos.values, index=tempos_formatados.Atividade).to_dict()


# --- Dicionários ---
MODELOS_CSV = {
    "Accelo": {"baumuster": ["C951102", "C951104", "C951111"], "perna": 1, "tempos": "Yamazumi - Accelo.csv"},
    "Atego": {"baumuster":["C951500", "C951501", "C951511", "C951514", "C951530", "C951544"],"perna": 1, "tempos":"Yamazumi - Atego.csv"},
    "Atego (ATP)":{"baumuster": ["C968403", "C968114"], "perna": 1, "tempos":"Yamazumi - ATP.csv"}, 
    "Actros":{"baumuster": ["C963400", "C963403", "C963411", "C963414", "C963424", "C963425"], "perna": 2, "tempos":"Yamazumi - Actros.csv"},
    "Arocs": {"baumuster":["C964016", "C964216", "C964231", "C964416"], "perna": 2, "tempos":"Yamazumi - Arocs.csv"},
    "Axor (ATP +)":{"baumuster": ["C968150", "C968450", "C968453", "C968461", "C968475"], "perna":2, "tempos": "Yamazumi - Actros.csv"}
}

POSTOS_ATIVIDADES = {
    "Accelo": {'30A':{'atividades': ['ARREFEC','DIESEL','REAPERTO'],'operadores': 2},'31A':{'atividades':['ESTEPE','MECANICA_1','MECANICA_2'], 'operadores':2},
               '32A':{'atividades':['5AA_RODA','ELACTRICA_1','ELACTRICA_2'], 'operadores':2}, '32C':{'atividades':['GRADE'], 'operadores':1},
                '33A':{'atividades':['PNEU_LD','PNEU_LE','APERTO_LD','APERTO_LE'],'operadores':2},'34A':{'atividades':['CONTROLE','MOTORISTA'],'operadores':2},
                '38':{'atividades':[],'operadores':0}},
    "Atego": {'30A':{'atividades': ['ARREFEC','DIESEL','REAPERTO'],'operadores': 2},'31A':{'atividades':['ESTEPE','MECANICA_1','MECANICA_2'], 'operadores':2},
               '32A':{'atividades':['5AA_RODA','ELACTRICA_1','ELACTRICA_2'], 'operadores':2}, '32C':{'atividades':['GRADE'], 'operadores':1},
                '33A':{'atividades':['PNEU_LD','PNEU_LE','APERTO_LD','APERTO_LE'],'operadores':2},'34A':{'atividades':['CONTROLE','MOTORISTA'],'operadores':2},
                '38':{'atividades':[],'operadores':0}},
    "Atego (ATP)":{'30A':{'atividades': ['ARREFEC','DIESEL','REAPERTO'],'operadores': 2},'31A':{'atividades':['ESTEPE','MECANICA_1','MECANICA_2'], 'operadores':2},
               '32A':{'atividades':['5AA_RODA','ELACTRICA_1','ELACTRICA_2'], 'operadores':2}, '32C':{'atividades':['GRADE'], 'operadores':1},
                '33A':{'atividades':['PNEU_LD','PNEU_LE','APERTO_LD','APERTO_LE'],'operadores':2},'34A':{'atividades':['CONTROLE','MOTORISTA'],'operadores':2},
                '38':{'atividades':[],'operadores':0}}, 
    "Actros":{"30B": {'atividades':["ABASTECIMENTO", "PASSADISSO"],'operadores':2}, "31B": {'atividades':["CHINELEIRA",'PARALAMA_LD','PARALAMA_LE'],'operadores':3},
              '32B':{'atividades':['5AA_RODA'],'operadores':2},'32D':{'atividades':['ELACTRICA_1','CONTROLE'],'operadores':2},
              '33B':{'atividades':['PNEU_LD','PNEU_LE'],'operadores':3}, '34B':{'atividades':['ELACTRICA_2','ELACTRICA_3','COLUNA5'],'operadores':2}, 
              '39':{'atividades':[],'operadores':0}},
    "Arocs": {"30B": {'atividades':["ABASTECIMENTO", "PASSADISSO"],'operadores':2}, "31B": {'atividades':["CHINELEIRA",'PARALAMA_LD','PARALAMA_LE'],'operadores':3},
              '32B':{'atividades':['5AA_RODA'],'operadores':2},'32D':{'atividades':['ELACTRICA_1','CONTROLE'],'operadores':2},
              '33B':{'atividades':['PNEU_LD','PNEU_LE'],'operadores':3}, '34B':{'atividades':['ELACTRICA_2','ELACTRICA_3','COLUNA5'],'operadores':2}, 
              '39':{'atividades':[],'operadores':0}},
    "Axor (ATP +)":{"30B": {'atividades':["ABASTECIMENTO", "PASSADISSO"],'operadores':2}, "31B": {'atividades':["CHINELEIRA",'PARALAMA_LD','PARALAMA_LE'],'operadores':3},
              '32B':{'atividades':['5AA_RODA'],'operadores':2},'32D':{'atividades':['ELACTRICA_1','CONTROLE'],'operadores':2},
              '33B':{'atividades':['PNEU_LD','PNEU_LE'],'operadores':3}, '34B':{'atividades':['ELACTRICA_2','ELACTRICA_3','COLUNA5'],'operadores':2}, 
              '39':{'atividades':[],'operadores':0}}
}

# DICIONÁRIO COM SOMA DOS TEMPOS POR POSTOS E TEMPOS POR OPERADOR
resultado = {}
for modelo, info in MODELOS_CSV.items():
    arquivo_csv = info["tempos"]
    tempos_atividades = get_process_times_from_csv(arquivo_csv)
    
    postos_dict = {}
    for posto, valor in POSTOS_ATIVIDADES[modelo].items():
        soma_tempo = sum(
            tempos_atividades.get(a.upper(), 0)
            for a in valor['atividades']
        )
        num_op = valor['operadores']
        tempo_op = soma_tempo / num_op if num_op != 0 else 0
        postos_dict[posto] = {
            'tempo_total': soma_tempo,
            'operador': num_op,
            'tempo_operador': tempo_op
        }
    
    resultado[modelo] = postos_dict

# --- Exibe resultado final ---
print("\n--- Tempos por posto por modelo ---")
for modelo, valor in resultado.items():
    print(f"{modelo}: {valor}")


--- Tempos por posto por modelo ---
Accelo: {'30A': {'tempo_total': 804, 'operador': 2, 'tempo_operador': 402.0}, '31A': {'tempo_total': 290, 'operador': 2, 'tempo_operador': 145.0}, '32A': {'tempo_total': 714, 'operador': 2, 'tempo_operador': 357.0}, '32C': {'tempo_total': 270, 'operador': 1, 'tempo_operador': 270.0}, '33A': {'tempo_total': 1108, 'operador': 2, 'tempo_operador': 554.0}, '34A': {'tempo_total': 531, 'operador': 2, 'tempo_operador': 265.5}, '38': {'tempo_total': 0, 'operador': 0, 'tempo_operador': 0}}
Atego: {'30A': {'tempo_total': 922, 'operador': 2, 'tempo_operador': 461.0}, '31A': {'tempo_total': 267, 'operador': 2, 'tempo_operador': 133.5}, '32A': {'tempo_total': 720, 'operador': 2, 'tempo_operador': 360.0}, '32C': {'tempo_total': 234, 'operador': 1, 'tempo_operador': 234.0}, '33A': {'tempo_total': 1246, 'operador': 2, 'tempo_operador': 623.0}, '34A': {'tempo_total': 541, 'operador': 2, 'tempo_operador': 270.5}, '38': {'tempo_total': 0, 'operador': 0, 'tempo_operado

ETAPA 2: OBTENDO DADOS SEQUENCIA DO DIA

In [19]:
sequencia = pd.read_csv("sequencia_producao.csv", sep=';')

# Extrai os 7 primeiros caracteres da coluna 'Baumuster'
sequencia['Baumuster_7dig'] = sequencia['Baumuster'].astype(str).str[:7]
sequencia_modelos = sequencia["Baumuster_7dig"].tolist()
print(sequencia_modelos)

['C951501', 'C951501', 'C968403', 'C968450', 'C951102', 'C963425', 'C951501', 'C968114', 'C951501', 'C951104', 'C951501', 'C963425', 'C951501', 'C968114', 'C951104', 'C963414', 'C963425', 'C963411', 'C963414', 'C951104', 'C968403', 'C951102', 'C963425', 'C951501', 'C951102', 'C951501', 'C968403', 'C963425', 'C951104', 'C951544', 'C951501', 'C963414', 'C951501', 'C951501', 'C951501', 'C963411', 'C951511', 'C951104', 'C951501', 'C963425', 'C951501', 'C951511', 'C951104', 'C963425', 'C951501', 'C968403', 'C951501', 'C963425', 'C951104', 'C968403', 'C951501', 'C963425', 'C951104', 'C951104', 'C963425', 'C951501', 'C951514', 'C963414', 'C951104', 'C951544', 'C951501', 'C951544', 'C951511', 'C963425', 'C951514', 'C951511', 'C963411', 'C951501', 'C968403', 'C951104', 'C963425', 'C951102', 'C951500', 'C951511', 'C963414', 'C951104', 'C951511', 'C951104', 'C963425', 'C951511', 'C951501', 'C964016', 'C951511', 'C951102', 'C968450', 'C951501', 'C963425', 'C951501', 'C968114', 'C951104', 'C951104'

ETAPA 3: OBTENDO QUANTOS MODELOS SÃO PRODUZIDOS POR PERNA POR TURNO

In [ ]:

TURN_DURATION = 8*60*60
TAKT_TIME =5.5*60
# Simulação
def simulacao_linha(env, sequencia_modelos, MODELOS_CSV, resultado):
    # Criar recursos (postos) para cada modelo e perna
    postos_perna1 = {}
    postos_perna2 = {}

    for modelo, postos in resultado.items():
        perna = MODELOS_CSV[modelo]["perna"]
        postos_dict = postos_perna1 if perna == 1 else postos_perna2
        for posto in postos:
            if posto not in postos_dict:
                postos_dict[posto] = simpy.Resource(env, capacity=1)

    # Variáveis de contagem de saída
    saidas = {"perna1": 0, "perna2": 0}

    # Função para processar um modelo
    def processar_modelo(env, nome, modelo_nome, perna, postos, recursos):
        for posto, dados in postos.items():
            tempo = dados["tempo_operador"]
            with recursos[posto].request() as req:
                yield req
                yield env.timeout(tempo)
        saidas[f"perna{perna}"] += 1

    # Função para lançar os modelos com base no takt time
    def alimentador(env):
        for i, baumuster in enumerate(sequencia_modelos):
            yield env.timeout(TAKT_TIME)

            modelo_nome = next((m for m, v in MODELOS_CSV.items() if baumuster in v["baumuster"]), None)
            if modelo_nome is None:
                continue

            perna = MODELOS_CSV[modelo_nome]["perna"]
            postos = resultado[modelo_nome]
            recursos = postos_perna1 if perna == 1 else postos_perna2
            env.process(processar_modelo(env, f"{modelo_nome}_{i}", modelo_nome, perna, postos, recursos))

    env.process(alimentador(env))
    env.run(until=TURN_DURATION)
    print(f"\nModelos saíram da perna 1: {saidas['perna1']}")
    print(f"Modelos saíram da perna 2: {saidas['perna2']}")
    # print(f'Total de:{saidas} modelos')
    return saidas


In [28]:
env = simpy.Environment()
simulacao_linha(env, sequencia_modelos, MODELOS_CSV, resultado)


Modelos saíram da perna 1: 44
Modelos saíram da perna 2: 22
Total de:{'perna1': 44, 'perna2': 22} modelos


{'perna1': 44, 'perna2': 22}

In [ ]:

def caminhao(env, id_caminhao, sequencia_modelos, fabrica, resultados):
    tempo_entrada = env.now
   
    # for modelo, info in MODELOS_CSV.items():
    #     for b in info["baumuster"]:
    #         if b in sequencia_modelos:
    #             dados.append({"baumuster": b, "Modelo": modelo, "Perna": info["perna"]})

    #     sequencia_atualizada = (pd.DataFrame(dados)).to_csv("sequencia_atualizada.csv", index=False)
    #     print(f"baumuster {b} pertence ao modelo {modelo} e vai para a Perna {dados['perna']}")
    

    modelos=[]
    pernas=[]
    for b in sequencia_modelos:
        for modelo, info in MODELOS_CSV.items():
            if b in info["baumuster"]:
                p = info['perna']
                if p == 1 :
                elif p == 2:

# !!! TRAVAMOS AQUI !!!!!!
  
# if modelo in ["Accelo", "Atego", "ATP"]: nome_linha = "Perna 1"
# elif modelo in ["Actros", "Arocs"]: nome_linha = "Perna 2"

#     else: return

#     print(f"[{format_time(env.now)}] CHEGADA: Caminhão_{id_caminhao} ({modelo}) entrou na fila da {nome_linha}.")

    operadores = fabrica['operadores'][nome_linha]
    linha_especifica = fabrica['linhas'][nome_linha]
    processos_modelo = PROCESSOS_FABRICA[modelo]
    
    tempo_total_processo = sum(processos_modelo.values())
    
    for nome_posto, tempo_proc in processos_modelo.items():
        if nome_posto not in linha_especifica or tempo_proc <= 0: continue
        
        posto = linha_especifica[nome_posto]
        
        with posto.request() as req_posto, operadores.request() as req_operador:
            yield req_posto & req_operador
            
            yield env.timeout(tempo_proc)

    tempo_saida = env.now
    lead_time = tempo_saida - tempo_entrada
    
    print(f"[{format_time(env.now)}] SAÍDA  : Caminhão_{id_caminhao} ({modelo}) finalizou. Lead Time: {format_time(lead_time)}")

    resultados.append({
        "ID": id_caminhao, "Modelo": modelo, "Linha": nome_linha,
        "LeadTime_min": lead_time / 60,
        "Tempo_Processo_min": tempo_total_processo / 60,
        "Tempo_Espera_min": (lead_time - tempo_total_processo) / 60,
        "Tempo_Saida_min": tempo_saida / 60,
    })


In [ ]:

# ETAPA 2: LÓGICA DA SIMULAÇÃO

# EXTRAINDO BAUMUSTERS DA LISTA SEQUENCIA DO DIA
sequencia = pd.read_csv("sequencia_producao.csv", sep=';')

# Extrai os 7 primeiros caracteres da coluna 'Baumuster'
sequencia['Baumuster_7dig'] = sequencia['Baumuster'].astype(str).str[:7]
sequencia_modelos = sequencia["Baumuster_7dig"].tolist()
print(sequencia_modelos)
#print(sequencia[['Baumuster', 'Baumuster_7dig']].head(20))

dados =[] 

def format_time(segundos):
    m, s = divmod(segundos, 60); h, m = divmod(m, 60)
    return f"{int(h):02d}:{int(m):02d}:{int(round(s,0)):02d}"

 
def caminhao(env, id_caminhao, sequencia_modelos, fabrica, resultados):
    tempo_entrada = env.now
   
    for modelo, info in MODELOS_CSV.items():
        for b in info["baumuster"]:
            if b in sequencia_modelos:
                dados.append({"baumuster": b, "Modelo": modelo, "Perna": info["perna"]})

        sequencia_atualizada = (pd.DataFrame(dados)).to_csv("sequencia_atualizada.csv", index=False)
        print(f"baumuster {b} pertence ao modelo {modelo} e vai para a Perna {dados['perna']}")
    
    if modelo in ["Accelo", "Atego", "ATP"]: nome_linha = "Perna 1"
    elif modelo in ["Actros", "Arocs"]: nome_linha = "Perna 2"

    else: return

    print(f"[{format_time(env.now)}] CHEGADA: Caminhão_{id_caminhao} ({modelo}) entrou na fila da {nome_linha}.")

    operadores = fabrica['operadores'][nome_linha]
    linha_especifica = fabrica['linhas'][nome_linha]
    processos_modelo = PROCESSOS_FABRICA[modelo]
    
    tempo_total_processo = sum(processos_modelo.values())
    
    for nome_posto, tempo_proc in processos_modelo.items():
        if nome_posto not in linha_especifica or tempo_proc <= 0: continue
        
        posto = linha_especifica[nome_posto]
        
        with posto.request() as req_posto, operadores.request() as req_operador:
            yield req_posto & req_operador
            
            yield env.timeout(tempo_proc)

    tempo_saida = env.now
    lead_time = tempo_saida - tempo_entrada
    
    print(f"[{format_time(env.now)}] SAÍDA  : Caminhão_{id_caminhao} ({modelo}) finalizou. Lead Time: {format_time(lead_time)}")

    resultados.append({
        "ID": id_caminhao, "Modelo": modelo, "Linha": nome_linha,
        "LeadTime_min": lead_time / 60,
        "Tempo_Processo_min": tempo_total_processo / 60,
        "Tempo_Espera_min": (lead_time - tempo_total_processo) / 60,
        "Tempo_Saida_min": tempo_saida / 60,
    })

def gerador_de_producao(env, fabrica, resultados, takt_time):
    id_caminhao = 0
    modelos_perna1 = [m for m in PROCESSOS_FABRICA.keys() if m in ["Accelo", "Atego", "ATP"]]
    modelos_perna2 = [m for m in PROCESSOS_FABRICA.keys() if m in ["Actros", "Arocs"]]

    while True:
        id_caminhao += 1
        
        if random.random() < 0.80 and modelos_perna1: 
            modelo_escolhido = random.choice(modelos_perna1)
        elif modelos_perna2:
            modelo_escolhido = random.choice(modelos_perna2)
        else:
            modelo_escolhido = random.choice(list(PROCESSOS_FABRICA.keys()))

        env.process(caminhao(env, id_caminhao, modelo_escolhido, fabrica, resultados))
        yield env.timeout(takt_time)



In [ ]:
# ETAPA 3: EXECUÇÃO E RELATÓRIOS

if not PROCESSOS_FABRICA:
    print("\n[ERRO FATAL] Nenhum dado foi carregado.")
else:
    # --- ESTRUTURA ---
    env = simpy.Environment()
    resultados_simulacao = []
    
    fabrica = {
        "operadores": {
            "Perna 1": simpy.Resource(env, capacity=NUM_OPERADORES_PERNA1),
            "Perna 2": simpy.Resource(env, capacity=NUM_OPERADORES_PERNA2)
        }, "linhas": {"Perna 1": {}, "Perna 2": {}}
    }
    
    postos_p1 = set().union(*(p.keys() for m, p in PROCESSOS_FABRICA.items() if m in ["Accelo", "Atego", "ATP"]))
    fabrica['linhas']['Perna 1'] = {posto: simpy.Resource(env, capacity=1) for posto in postos_p1}
    postos_p2 = set().union(*(p.keys() for m, p in PROCESSOS_FABRICA.items() if m in ["Actros", "Arocs"]))
    fabrica['linhas']['Perna 2'] = {posto: simpy.Resource(env, capacity=1) for posto in postos_p2}
    
    # --- EXECUÇÃO ---
    print("\n--- Iniciando Simulação: Cenário 'As-Is' Real ---")
    print(f"Takt Time: {TAKT_TIME_SEGUNDOS/60:.2f} min | Perna 1: {NUM_OPERADORES_PERNA1} op. | Perna 2: {NUM_OPERADORES_PERNA2} op.")
    
    env.process(gerador_de_producao(env, fabrica, resultados_simulacao, TAKT_TIME_SEGUNDOS))
    env.run(until=HORAS_SIMULACAO * 3600)
    
    print("\n" + "="*60)
    print("      RELATÓRIO GERENCIAL DA SIMULAÇÃO (AS-IS)")
    print("="*60)

    # --- ANÁLISE DOS RESULTADOS ---
    df_resultados = pd.DataFrame(resultados_simulacao)
    
    if df_resultados.empty:
        print("\nNenhum caminhão completou a produção.")
    else:
        # 1. Análise da Meta de Produção
        df_meta_batida = df_resultados[df_resultados['Tempo_Saida_min'] <= HORAS_META * 60]
        qtd_na_meta = len(df_meta_batida)
        
        print("\n--- 1. ANÁLISE DA META DE PRODUÇÃO (c/ 'Fator Cagaço') ---")
        if qtd_na_meta >= META_PRODUCAO:
            print(f" SUCESSO! A meta foi batida, com {qtd_na_meta} caminhões produzidos nas primeiras {HORAS_META} horas.")
        else:
            print(f" FALHA! Apenas {qtd_na_meta} de {META_PRODUCAO} caminhões foram produzidos nas primeiras {HORAS_META} horas.")

        # 2. Análise de Desempenho
        print("\n--- 2. DESEMPENHO POR PERNA ---")
        desempenho = df_resultados.groupby('Linha').agg(
            Qtd_Produzida=('ID', 'count'),
            Lead_Time_Medio_min=('LeadTime_min', 'mean'),
            Tempo_Medio_Espera_min=('Tempo_Espera_min', 'mean')
        ).round(2)
        print(desempenho)

        # 3. Novo Relatório Visual
        print("\n--- 3. Gerando Relatório Visual Gerencial ---")
        fig, axes = plt.subplots(1, 2, figsize=(18, 8), gridspec_kw={'width_ratios': [1, 1]})
        fig.suptitle('Relatório Gerencial da Simulação (Cenário As-Is Real)', fontsize=20)

        # Gráfico 1: Produção por Perna
        prod_counts = df_resultados['Linha'].value_counts().reindex(['Perna 1', 'Perna 2']).fillna(0)
        sns.barplot(x=prod_counts.index, y=prod_counts.values, ax=axes[0], palette='viridis', hue=prod_counts.index, legend=False)
        axes[0].set_title('Produção Total por Perna', fontsize=16)
        axes[0].set_ylabel('Qtd. de Caminhões Produzida')
        axes[0].set_ylim(0, prod_counts.max() * 1.15)
        for i, v in enumerate(prod_counts.values):
            axes[0].text(i, v, str(int(v)), ha='center', va='bottom', fontsize=14)
        
        # Gráfico 2: Lead Time por Perna
        sns.boxplot(x='Linha', y='LeadTime_min', data=df_resultados, ax=axes[1], palette='coolwarm', hue='Linha', order=['Perna 1', 'Perna 2'], legend=False)
        axes[1].set_title('Lead Time por Perna', fontsize=16)
        axes[1].set_ylabel('Tempo Total de Produção (minutos)')
        
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.savefig('relatorio_gerencial_real.png')
        print(" SUCESSO: Gráfico 'relatorio_gerencial_real.png' salvo.")